**Impetum Diem Tempus Fugit** or "Attack the day because time flies"

This is a small Python project but it covers a lot of concepts. My Python and problem solving skills are on display here.

As the name of my project implies, it has to do with time.  I imported datetime which recognizes Daylight Savings Time in local time.  However that time is not monotonic so I have to adjust the code to get an accurate measure of time.  Using my code, I can put in a datetime, enter a certain amount of time, going forward or backwards, and come up with a new datetime. I can also input two dates and get the total amount of elapsed time between the two.  All results reflect the actual time that has passed, regardless of DST.  

The first problem I faced was how to handle Daylight Savings Time in March.  On Daylight Savings Day in March the time at 2AM jumps to 3AM.  Therefore we only have 23 hours on that day and time between 2AM and 3AM does not exist.  To start with, I entered a validation step into my constructor method.  I created my naive_dt object and assigned this to my mountain timezone.  This gave me my aware_dt object.  I converted that to UTC.  Then I converted that back to roundtrip_local time.  For my validation check I used an "if" statement to check to see if my roundtrip_local time was equal to my original naive_dt time.  If it wasn't, that meant I was dealing with a non-existant time and the ValueError kicked in.  Here, at the last minute, it has crossed my mind, to use a @staticmethod decorator for my validation code.  It is something that I will probably add to the code at a later date.

In November, it's a different situation.  On Daylight Savings Day in November at 2AM, the clock goes backwards to 1AM and we relive that hour over again.  So that means that we experience 25 hours on that day.  In coding for the "fall back' I had to use a "fold" argument.  When I enter 1:30AM on November 2, 2025, is it the first time in the 1AM hour or the second time around?  If the fold=0 this mean it's the first time around.  fold=1 stands for the second time.  I only enter the keyword for the fold, as 0 or 1, if I input the 1AM hour on November 2, 2025(or the date, future or past, of a November time change). 

I'm using Object Oriented Programming with my class Tempus.  I used an alternate constructor, from_datetime, with the @classmethod decorator.  This gave me more flexibility.  Especially using keys for the timezones and the fold.  Finally, I also used dunders that included the use of __repr__ instead of __str__.  It's a little more professional and better suited to development. 

In [2]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

class Tempus:
    def __init__(self, year, month, day, hour, minute, second, timezone_str="America/Denver", fold=0):
        self.timezone = ZoneInfo(timezone_str)
        naive_dt = datetime(year, month, day, hour, minute, second)
        aware_dt = naive_dt.replace(tzinfo=self.timezone, fold=fold)

        """Validation that the time exists in the specified timezone"""
        utc_dt = aware_dt.astimezone(ZoneInfo("UTC"))
        roundtrip_local = utc_dt.astimezone(self.timezone)

        if (roundtrip_local.hour != hour or
            roundtrip_local.minute != minute or
            roundtrip_local.second != second):
            raise ValueError(f"The time {hour}:{minute}:{second} on {year}-{month}-{day} does not exist in timezone {timezone_str} due to DST transition.")

        self.datetime = aware_dt

    def __add__(self, delta):
        """On both the add and subtract-I converted time to UTC for the calculation. Then converted back to wall time"""
        if isinstance(delta, timedelta):
            utc_time = self.datetime.astimezone(ZoneInfo("UTC")) + delta
            return Tempus.from_datetime(utc_time.astimezone(self.timezone))
        return NotImplemented

    def __sub__(self, other):
        if isinstance(other, Tempus):
            utc_self = self.datetime.astimezone(ZoneInfo("UTC"))
            utc_other = other.datetime.astimezone(ZoneInfo("UTC"))
            return utc_self - utc_other
        elif isinstance(other, timedelta):
            utc_time = self.datetime.astimezone(ZoneInfo("UTC")) - other
            return Tempus.from_datetime(utc_time.astimezone(self.timezone))
        return NotImplemented

    def __repr__(self):
        """Formatted to the date and time format used in the United States"""
        return self.datetime.strftime("US Format Mountain Time: %m-%d-%Y %I:%M:%S %p %Z%z")

    @classmethod
    def from_datetime(cls, dt):
        return cls(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, dt.tzinfo.key, dt.fold)
    
    
tempus1 = Tempus(2025,1,1, 22,1,1)
tempus2 = tempus1 + timedelta(days=90)
print("Adding 90 days to January 1, 2025 at 10:01:01PM: ",tempus2)
print("The answer is April 1 at 11:01:01PM.  The additional hour is because of DST")
print("\n")

tempus3 = Tempus(2025,10,1,22,1,1)
tempus4 = tempus3 + timedelta(days=92)
print("Adding 90 days to November 1, 2025 at 10:01:01PM: ",tempus4)
print("The answer is January 1, 2026 at 9:01:01PM.  The subtracted hour is because of DST")
print("\n")

tempus5 = Tempus(2025,3,9,1,1,1)
tempus6 = tempus5 + timedelta(hours=24)
print("Adding 24 hours to March 9, 2025 at 1:01:01AM: ",tempus6)
print("The answer is March 10, 2026 at 2:01:01AM.  The additional hour the following day, March 10, 2025 is because of DST")
print("\n")

tempus7 = Tempus(2025,11,2,1,1,1,fold=0)
tempus8 = tempus7 + timedelta(hours=24)
print("Adding 24 hours to November 2, 2025 at 1:01:01AM using fold=0: ",tempus8)
print("The answer is March 10, 2026 at 12:01:01AM.  Using fold=0,The subtracted hour in the following day, November 3, 2025 is because of DST")
print("\n")

tempus9 = Tempus(2025,11,2,1,1,1,fold=1)
tempus10 = tempus9 + timedelta(hours=24)
print("Adding 24 hours to November 2, 2025 at 1:01:01AM using fold=1: ",tempus10)
print("The answer is March 10, 2026 at 12:01:01AM.  Using fold=1,The following day, November 3, 2025 time is the same")
print("\n")

tempus11 = Tempus(2025,1,1,1,1,1)
tempus12 = Tempus(2025,4,11,1,1,1)
tempus13 = tempus12-tempus11
print("The Timedelta between these two dates: ",tempus13)
print("The answer is 99 days and 23 hours.  Once again the difference is because of DST")


Adding 90 days to January 1, 2025 at 10:01:01PM:  US Format Mountain Time: 04-01-2025 11:01:01 PM MDT-0600
The answer is April 1 at 11:01:01PM.  The additional hour is because of DST


Adding 90 days to November 1, 2025 at 10:01:01PM:  US Format Mountain Time: 01-01-2026 09:01:01 PM MST-0700
The answer is January 1, 2026 at 9:01:01PM.  The subtracted hour is because of DST


Adding 24 hours to March 9, 2025 at 1:01:01AM:  US Format Mountain Time: 03-10-2025 02:01:01 AM MDT-0600
The answer is March 10, 2026 at 2:01:01AM.  The additional hour the following day, March 10, 2025 is because of DST


Adding 24 hours to November 2, 2025 at 1:01:01AM using fold=0:  US Format Mountain Time: 11-03-2025 12:01:01 AM MST-0700
The answer is March 10, 2026 at 12:01:01AM.  Using fold=0,The subtracted hour in the following day, November 3, 2025 is because of DST


Adding 24 hours to November 2, 2025 at 1:01:01AM using fold=1:  US Format Mountain Time: 11-03-2025 01:01:01 AM MST-0700
The answer is March 